One of the potential issues we have uncovered is whether the GFDN is able to model fade-in behaviour that is observed in coupled rooms when there is no line of sight between the source and the listener. In general, fade-in behaviour occurs when we convolve two exponential decays. Let two broadband EDCs be represented as,

$$h_{R1}(t) = A_1 e^{-\delta_1 t}, h_{R2}(t) = A_2 e^{-\delta_2 t}$$.
where $A_1, A_2$ are the initial amplitudes and $\delta_1$ and $\delta_2$ are the decay rates.
When we convolve the two EDCs, (which is what happens when the rooms are coupled), we get

$$
h_{R1, R2}(t) = \frac{A_1 A_2}{\delta_2 - \delta_1} \left(e^{-\delta_1 t} - e^{-\delta_2 t} \right)
$$
The transfer function of this output can be implemented with a biquad:
$$
H(z) = \frac{A_1 A_2} {\delta_2 - \delta_1} \left[\frac{(e^{-\delta_1} - e^{-\delta_2})z^{-1}}{1 - (e^{-\delta_1} + e^{-\delta_2})z^{-1} + e^{-(\delta_1 + \delta_2)} z^{-2}} \right]
$$
Let's plot the envelope for a range of decay rates.

In [ ]:
import os
import numpy as np
from typing import List
from numpy.typing import ArrayLike, NDArray
from diff_gfdn.utils import db, db2lin, ms_to_samps, is_unitary
from diff_gfdn.analysis import calculate_energy_envelope
from scipy.signal import sosfilt
import matplotlib.pyplot as plt
from pathlib import Path
os.chdir('..')  # This changes the working directory to DiffGFDN
%matplotlib inline


def reverb_time_to_time_constant(rt60_ms:float, fs:float):
    """convert time in ms to energy decay slope"""
    return (np.array(rt60_ms) * 1e-3) / np.log(0.001)

def convolve_exponential_decay(init_amps : List, rt60_ms: List, ir_len_ms: int, fs:float) -> ArrayLike:
    """
    Convolve two exponential decays
    Args:
        init_amps (list): list of initial amplitudes
        rt60_ms (list): list of reverb times in ms
        ir_len_ms (int): length of EDC to plot
        fs (float): sample rate
    Returns:
        ArrayLike: the output of convolution of two exponentials
    """
    ir_len_samps = ms_to_samps(ir_len_ms, fs)
    decay_slope = 1.0 / reverb_time_to_time_constant(rt60_ms, fs)
    time_vector = np.arange(0, float(ir_len_samps)/fs, 1.0/fs)
    assert len(init_amps) == len(decay_slope)
    assert len(decay_slope) == 2, "This function only works for 2 slopes"
    output = np.prod(init_amps) / (decay_slope[0] - decay_slope[-1]) * \
             (np.exp(decay_slope[0]*time_vector) - np.exp(decay_slope[-1]*time_vector))
    return time_vector, output

def convolve_exponential_with_biquad(init_amps : List, rt60_ms: List, ir_len_ms: int, fs:float):
    """Convolve two exponential decays using a biquad"""
    decay_slope = 1.0 / reverb_time_to_time_constant(rt60_ms, fs)
    const = np.prod(init_amps) / (decay_slope[0] - decay_slope[-1])
    b = const * np.array([0, np.exp(decay_slope[0]/fs) - np.exp(decay_slope[-1]/fs), 0.0])
    a = np.array([1.0, -(np.exp(decay_slope[0]/fs) + np.exp(decay_slope[1]/fs)), np.exp(np.sum(decay_slope)/fs)])
    sos = np.concatenate((b, a), axis=0)
    
    ir_len_samps = ms_to_samps(ir_len_ms, fs)
    inp = np.zeros(ir_len_samps)
    inp[0] = 1.0
    time_vector = np.arange(0, float(ir_len_samps)/fs, 1.0/fs)
    output = sosfilt(sos, inp)
    return time_vector, output
    

In [ ]:
fig_path = Path('figures/').resolve()
init_amps = [1.0, 1.0]
fs = 48000
ir_len_ms = 1000

# plot for a variety of shorter RT60s
rt60_long = 500
rt60_short = np.arange(50, rt60_long, 50)

plt.figure()
for k in range(len(rt60_short)):
    rt60_ms = [rt60_short[k], rt60_long]
    time_vector, output = convolve_exponential_decay(init_amps, rt60_ms, ir_len_ms, fs)
    # time_vector, output = convolve_exponential_with_biquad(init_amps, rt60_ms, ir_len_ms, fs)
    plt.plot(time_vector, db(output), label=f'Short rt60={rt60_short[k]}')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (dB)')
plt.legend()
plt.savefig(f'{fig_path}/fade_in_vary_short_decay.png')

# plot for a variety of long RT60s
rt60_long = np.arange(500, 1000, 50)
rt60_short = 100
plt.figure()
for k in range(len(rt60_long)):
    rt60_ms = [rt60_short, rt60_long[k]]
    time_vector, output = convolve_exponential_decay(init_amps, rt60_ms, ir_len_ms, fs)
    plt.plot(time_vector, db(output), label=f'Long rt60={rt60_long[k]}')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (dB)')
plt.legend()
plt.savefig(f'{fig_path}/fade_in_vary_long_decay.png')


# plot for a variety of initial amplitudes
rt60_long = 500
rt60_short = 100
init_amps_short = np.arange(0.1, 1, 0.2)
init_amps_long = 1.0
plt.figure()
for k in range(len(init_amps_short)):
    rt60_ms = [rt60_short, rt60_long]
    init_amps = [init_amps_short[k], init_amps_long]
    time_vector, output = convolve_exponential_decay(init_amps, rt60_ms, ir_len_ms, fs)
    plt.plot(time_vector, db(output), label=f'Short init amp={init_amps_short[k]:.3f}')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (dB)')
plt.legend()


# plot for a variety of initial amplitudes
init_amps_long = np.arange(0.1, 1, 0.2)
init_amps_short = 1.0
plt.figure()
for k in range(len(init_amps_long)):
    rt60_ms = [rt60_short, rt60_long]
    init_amps = [init_amps_short, init_amps_long[k]]
    time_vector, output = convolve_exponential_decay(init_amps, rt60_ms, ir_len_ms, fs)
    plt.plot(time_vector, db(output), label=f'Long init amp={init_amps_long[k]:.3f}')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (dB)')
plt.legend()

From the above plots, we see that the fade-in slope depends on the shorter reverberation time, and the fade-out slope depends on the longer reverberation time.

The question we want to answer is, can the GFDN model this fade in property? To determine this consider the simplest GFDN with two delay lines, each of length $\tau_1, \tau_2$ samples, with gains per sample, $\Gamma_1, \Gamma_2$. They are connected via a $2 \times 2$ unitary feedback matrix $\mathbf{A}$, and input, output gains, $\mathbf{b}, \mathbf{c} \in \mathbb{R}^{2 \times 1}$ respectively. The transfer function of this GFDN is given by,

$$
\begin{aligned}
H(z) &= \mathbf{c}^T \left(\mathbf{D_m}^{-1}(z) \mathbf{\Gamma}^{-1} - \mathbf{A} \right)^{-1} \mathbf{b} \\
H(z) &= \frac{\mathbf{c}^T \text{adj} \left(\mathbf{D_m}^{-1}(z) \mathbf{\Gamma}^{-1} - \mathbf{A} \right)}{\text{det} \left( \mathbf{D_m}^{-1}(z) \mathbf{\Gamma}^{-1} - \mathbf{A} \right)} \\
 &= \begin{bmatrix} c_1 & c_2
\end{bmatrix} 
\left(
\begin{bmatrix}\frac{z^{\tau_1}}{\Gamma_1} & 0 \\
0 & \frac{z^{\tau_2}}{\Gamma_2}
\end{bmatrix} - 
\begin{bmatrix}
A_{11} & A_{12} \\
A_{21} & A_{22}
\end{bmatrix}
\right)^{-1}
\begin{bmatrix}
b_1 \\
b_2
\end{bmatrix}
\end{aligned}
$$

On simplifying this, we get,

$$
H(z) = \frac{z^{-\tau_1} c_1 b_1 \gamma_1 + z^{-\tau_2} c_2 b_2 \gamma_2 - z^{-\tau_1 - \tau_2}\gamma_1 \gamma_2 \ \mathbf{c}^T \text{adj}(\mathbf{A})\mathbf{b}}{1 + \gamma_1 \gamma_2 z^{-\tau_1 - \tau_2} - (A_{11}\gamma_1 z^{-\tau_1} + A_{22}\gamma_2 z^{-\tau})}
$$
We can simplify this further because $\text{adj}(\mathbf{A}) = \text{det}(\mathbf{A}) \mathbf{A}^T$. The transfer function becomes:
$$
\begin{aligned}
H(z) &= \frac{z^{-\tau_1} c_1 b_1 \gamma_1 + z^{-\tau_2} c_2 b_2 \gamma_2 - z^{-\tau_1 - \tau_2}\gamma_1 \gamma_2 \ \text{det}(\mathbf{A})  \mathbf{c}^T \mathbf{A}^T\mathbf{b}}{1 + \gamma_1 \gamma_2 z^{-\tau_1 - \tau_2} \text{det}(\mathbf{A}) - (A_{11}\gamma_1z^{-\tau_1} + A_{22}\gamma_2 z^{-\tau_2})} \\
&= \frac{ \mathbf{c}^T \mathbf{D_\gamma}(z) \mathbf{b} - \text{det}(\mathbf{A D_\gamma}(z)) \mathbf{c}^T \mathbf{A}^T\mathbf{b}}{1  - \text{tr}(\mathbf{A D_\gamma}(z)) +\text{det}(\mathbf{D_\gamma}(z))\text{det}(\mathbf{A})} \\
&= \frac{\mathbf{c}^T \left(\mathbf{D_\gamma}(z) -  \text{det}(\mathbf{A D_\gamma}(z)) \ \mathbf{A}^T \right) \mathbf{b}}{1  - \text{tr}(\mathbf{A D_\gamma}(z)) +\text{det}(\mathbf{A D_\gamma}(z))}
\end{aligned}
$$
where $\mathbf{D_\gamma}(z) = \text{diag}(\gamma_1 z^{-\tau_1}, \gamma_2 z^{-\tau_2})$ is the diagonal matrix of decays.

In [ ]:
from scipy.linalg import qr, hadamard, eig
from scipy.signal import impulse, tf2zpk
from scipy.fft import irfft, rfftfreq
import sympy as sp
import torch

def get_coprime_delay_lengths(delay_range_ms: List, sample_rate:float, num_delay_lines: int) -> List[int]:
    """Co-prime delay line lenghts for a given range"""
    np.random.seed(1234)
    delay_range_samps = ms_to_samps(np.asarray(delay_range_ms),
                                    sample_rate)
    # generate prime numbers in specified range
    prime_nums = np.array(list(
        sp.primerange(delay_range_samps[0], delay_range_samps[1])),
                          dtype=np.int32)
    rand_primes = prime_nums[np.random.permutation(len(prime_nums))]
    # delay line lengths
    delay_lengths = np.array(np.r_[rand_primes[:num_delay_lines - 1],
                                   sp.nextprime(delay_range_samps[1])],
                             dtype=np.int32)
    return delay_lengths


def get_abs_gain_from_rt60(rt60_ms: List, taus:List[int], fs: float):
    """Get delay line gains from the desired T60"""
    gains = db2lin(-60 * taus /(fs * rt60_ms * 1e-3))
    return gains
    
def get_random_unitary_matrix(N: int) -> NDArray:
    """Get a random unitary matrix of size N x N"""
    # start with a random matrix and get its QR decomposition
    (Q, R) = qr(np.random.randn(N, N))
    return np.matmul(Q, np.diag(np.sign(np.diag(R))))

def fill_diagonal(mat_2D : NDArray):
    """Fill the diagonal of an NxNxK times matrix with a NxK matrix"""
    num_del_lines = mat_2D.shape[0]
    num_freq_samps = mat_2D.shape[-1]
    mat_3D = np.zeros((num_del_lines, num_del_lines, num_freq_samps), dtype=mat_2D.dtype)
    for k in range(num_freq_samps):
        np.fill_diagonal(mat_3D[...,k], mat_2D[:,k])
    return mat_3D
    
def simple_gfdn_transfer_function(taus: List[int], gamma:List, A: NDArray, b: NDArray, c: NDArray):
    """
    Return the transfer function of a simple GFDN consisting of two delay lines connected with a 2x2
    unitary matrix with 2 different decay rates
    """
    bfilt = np.zeros(np.sum(taus)+1)
    afilt = np.zeros_like(bfilt)
    bfilt[taus[0]] = c[0,0] * gamma[0] * b[0,0]
    bfilt[taus[1]] = c[1,0] * gamma[1] * b[1,0]
    bfilt[taus[0] + taus[1]] = -np.prod(gamma) * np.linalg.det(A) * np.squeeze(c.T @ A.T @ b)
    afilt[0] = 1.0
    afilt[taus[0]] = -A[0,0] * gamma[0]
    afilt[taus[1]] = -A[1,1] * gamma[1]
    afilt[taus[0] + taus[1]] = np.prod(gamma) * np.linalg.det(A)
   
    zeros,poles,gains = tf2zpk(bfilt, afilt)
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    ax.plot(np.angle(zeros), np.abs(zeros), 'o')
    ax.plot(np.angle(poles), np.abs(poles), 'x')
    ax.set_rmax(1.1)
    ax.set_rticks([0.25, 0.5, 1])  # Less radial ticks
    ax.set_rlabel_position(-22.5)  # Move radial labels away from plotted line
    ax.grid(True)
    plt.show()

    return bfilt, afilt

def simple_gfdn_impulse_response(taus: List[int], 
                                 A: NDArray, 
                                 b: NDArray, 
                                 c: NDArray, 
                                 rt60_ms: List, 
                                 ir_len_ms: int, 
                                 fs: float, 
                                 method:str='safe'):
    """
    Return the impulse response of a simple GFDN consisting of two delay lines connected with a 2x2
    unitary matrix with 2 different decay rates
    """        
    gamma = get_abs_gain_from_rt60(rt60_ms, taus, fs)
    num_del_lines = len(taus)
    ir_len_samps = ms_to_samps(ir_len_ms, fs)
    time_vector = np.arange(0, float(ir_len_samps)/fs, 1.0/fs)

    # transfer function with frequency sampling method
    num_freq_samps = 2**int(np.ceil(np.log2(ir_len_samps)))
    z = np.ones(num_freq_samps) * np.exp(1j*np.linspace(0, np.pi, num_freq_samps))
    delays = np.repeat(z[np.newaxis,:], num_del_lines, axis=0) ** -taus[:, np.newaxis]
    freqs = rfftfreq(2*num_freq_samps-1, d=1.0/fs)
    A_expanded = np.repeat(A[...,np.newaxis], num_freq_samps, axis=-1)

    # direct computation from transfer function
    if method == 'safe':
        Ddecay_inv = fill_diagonal((1.0 / delays) / gamma[:, np.newaxis])
        P = Ddecay_inv - A_expanded
        # frequencies should be along last axis for proper inversion
        P_inv = np.linalg.inv(P.transpose(-1, 0, 1)).transpose(1, -1, 0)
        H = np.squeeze(np.einsum('mi, ijk, jm -> mk', c.T, P_inv, b))
    # alternate expression (only valid for 2x2 case)
    else:
        Ddecay = fill_diagonal(delays * gamma[:, np.newaxis])
        # this is of shape 1 x num_freq_samps
        det_D = np.linalg.det(Ddecay.transpose(-1,0,1))[np.newaxis, :]
        det_A = np.full((1, num_freq_samps), np.linalg.det(A))
        numerator = np.einsum('mi, ijk, jm -> mk', c.T, Ddecay - det_D*det_A*A_expanded.transpose(1, 0, -1), b)
        denominator = np.ones((1, num_freq_samps), dtype=np.complex64) - \
                      np.trace(np.einsum('ijk, jpk -> ipk', Ddecay, A_expanded), axis1=0, axis2=1) + det_D*det_A
        H = np.squeeze(numerator / denominator)

    h = irfft(H)
    return time_vector, h[:ir_len_samps]

In [ ]:
num_del_lines = 2
taus = get_coprime_delay_lengths([5, 10], fs, num_del_lines)

rt60_ms = np.array([100, 500])
decay_slope = 1.0 / reverb_time_to_time_constant(rt60_ms, fs)
gain_const = 1.0/(decay_slope[0] - decay_slope[-1])

ir_len_ms = 1000
time_vector, exp_edc = convolve_exponential_decay([1, 1], rt60_ms, ir_len_ms, fs)

# everything chosen randomly
b = np.random.randn(num_del_lines, 1)
c = np.random.randn(num_del_lines, 1)
A = get_random_unitary_matrix(num_del_lines)
assert np.allclose(A.T @ A, np.eye(num_del_lines))

# plot poles and zeros of FDN
# gamma = get_abs_gain_from_rt60(rt60_ms, taus, fs)
# bfilt, afilt = simple_gfdn_transfer_function(taus, gamma, A, b, c)

time_vector, output = simple_gfdn_impulse_response(taus, A, b, c, rt60_ms, ir_len_ms, fs)
plt.figure()
plt.plot(time_vector, db(output))
plt.plot(time_vector, db(exp_edc))
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (dB)')
plt.title('GFDN RIR with random parameterisation')

# to get an exact match
b = np.sqrt(np.abs(gain_const)) * np.array([1.0, 1.0])[:, np.newaxis]
c = np.sqrt(np.abs(gain_const)) * np.array([1.0, -1.0])[:, np.newaxis]
taus = np.array([taus[0], taus[0]])
A = np.eye(num_del_lines)
time_vector, output = simple_gfdn_impulse_response(taus, A, b, c, rt60_ms, ir_len_ms, fs)

plt.figure()
plt.plot(time_vector, db(output))
plt.plot(time_vector, db(exp_edc))
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (dB)')
plt.title('GFDN RIR with parameterisation to produce fade-in behaviour')

To achieve the exact fade-in characteristics obtained by convolving two exponentials with decay rates $\delta_1, \delta_2$, we must ensure that:
\begin{aligned}
\mathbf{b} &= \sqrt{\frac{1}{|\delta_2 - \delta_1|}}\begin{bmatrix} 1 & 1\end{bmatrix}^T \\
\mathbf{c} &= \sqrt{\frac{1}{|\delta_2 - \delta_1|}}\begin{bmatrix} 1 & -1\end{bmatrix}^T \\
\mathbf{D_m}(z) &= \begin{bmatrix} z^{-\tau} & 0 \\ 0 &  z^{-\tau}  \end{bmatrix} \\
\mathbf{\Gamma} &= \begin{bmatrix} 10^{-\frac{3  \tau}{T_{60_1} f_s}} & 0 \\ 0 & 10^{-\frac{3  \tau}{f_s  T_{60_2}}}  \end{bmatrix}, \quad T_{60} = \frac{- \ln(0.001)}{\delta} \\
\mathbf{A} &=  \begin{bmatrix} 1 & 0 \\ 0 & 1  \end{bmatrix}
\end{aligned}
This is effectively the same as subtracting the outputs of 2 FDNs, having all the same parameters, but different decay rates. This implies that the modes of both the groups have the same frequencies, but have different decay rates. When we subtract the modes, we get fade-in behaviour. 

Now, for a GFDN with $2$ groups, and $N$ delay lines per group, $\mathbf{A} = \mathbf{I}_{2N}$, and the input-output gains must satisfy,

\begin{aligned}
\mathbf{c}_N^T \mathbf{b}_N &= \frac{1}{|\delta_2 - \delta_1|} \\
\mathbf{c} &= \begin{bmatrix} \mathbf{c}_N \\ -\mathbf{c}_N\end{bmatrix}, \quad \mathbf{b} = \begin{bmatrix} \mathbf{b}_N \\ \mathbf{b}_N\end{bmatrix}
\end{aligned}

One way of ensuring this is to select, $\mathbf{c}_N = \frac{1}{|\delta_2 - \delta_1|} \frac{\mathbf{b}_N}{||\mathbf{b}_N||^2} \  \forall \ \mathbf{b} \in \mathbb{R}^{N \times 1}$. This parameterisation avoids any coupling between the two FDNs. In fact, if we change the feedback matrix, $\mathbf{A}$, symmetrically for both the groups such that the overall coupled feedback matrix is block-diagonal, the fade-in behaviour remains the same, and we have higher echo density but the amplitude at fade-in time changes.

In [ ]:
num_delay_lines_per_group = 4
num_groups = 2
num_del_lines = num_groups * num_delay_lines_per_group
# different taus in each group
# taus = get_coprime_delay_lengths([5, 10], fs, num_del_lines)

# use the same taus in each group
taus = get_coprime_delay_lengths([5, 10], fs, num_delay_lines_per_group)
taus = np.concatenate((taus, taus), axis=0)

decay_slope = 1.0 / reverb_time_to_time_constant(rt60_ms, fs)
gain_const = 1.0/(decay_slope[0] - decay_slope[-1])
rt60_ms_exp = np.concatenate((rt60_ms[0] * np.ones(num_delay_lines_per_group), rt60_ms[1] * np.ones(num_delay_lines_per_group)), axis=0)
ir_len_ms = 500
time_vector, exp_edc = convolve_exponential_decay([1, 1], rt60_ms, ir_len_ms, fs)

# to get an exact match
b = np.random.randn(num_delay_lines_per_group)
c = (gain_const / np.linalg.norm(b)**2) * b
b = np.concatenate((b, b))[:, np.newaxis]
c = np.concatenate((c, -c))[:, np.newaxis]
A = np.eye(num_del_lines)

time_vector, output = simple_gfdn_impulse_response(taus, A, b, c, rt60_ms_exp, ir_len_ms, fs)
envelope = calculate_energy_envelope(output, fs, smooth_time_ms=50)

plt.figure()
plt.plot(time_vector, db(output))
plt.plot(time_vector, db(exp_edc))
plt.plot(time_vector, db(envelope))
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (dB)')
plt.title('RIR with diagonal feedback matrix')

#try with a different intra-group feedback matrix
A = get_random_unitary_matrix(num_delay_lines_per_group)
A_block = np.zeros((num_del_lines, num_del_lines))
A_block[:num_delay_lines_per_group, :num_delay_lines_per_group] = A
A_block[num_delay_lines_per_group:, num_delay_lines_per_group:] = A
        
time_vector, output = simple_gfdn_impulse_response(taus, A_block, b, c, rt60_ms_exp, ir_len_ms, fs)
envelope = calculate_energy_envelope(output, fs, smooth_time_ms=50)

plt.figure()
plt.plot(time_vector, db(output))
plt.plot(time_vector, db(exp_edc))
plt.plot(time_vector, db(envelope))
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (dB)')
plt.title('RIR with symmetric block diagonal feedback matrix')

Let's also consider what happens when we perturb the matrices of the two groups slightly, i.e,
$$
\mathbf{A} = \begin{bmatrix}\mathbf{A}_1 & \mathbf{0} \\ \mathbf{0} & \mathbf{A}_2 \end{bmatrix}
$$
Now, each mode will have the following expression:

\begin{aligned}
h_m(t) &= e^{-\delta_1 t} e^{j\omega_1t} - e^{-\delta_2 t} e^{j\omega_2 t} \\
&= r(t) e^{j\phi (t)} \\
r(t) &= \sqrt{e^{-2\delta_1 t} + e^{-2 \delta_2 t} - 2e^{-t(\delta_1 + \delta_2)} \cos \left((\omega_2 - \omega_1)t\right)} \\
\phi(t) &= \arctan{\left(\frac{e^{-t \delta_1} \sin\omega_1 t - e^{-t \delta_2} \sin\omega_2 t}{e^{-t \delta_1} \cos\omega_1 t - e^{-t \delta_2} \cos\omega_2 t} \right)}
\end{aligned}
If $\omega_1 = \omega_2$, then we have $h_m(t) = e^{-t\delta_1} - e^{-t \delta_2}$. The fade-in time and peak value at fade-in time are affected by the perturbation. However, the decay rate post fade-in remains unaffected.

To replicate this behaviour in the GFDN, we use the following block diagonal matrix,
\begin{aligned}
\mathbf{A} &= \begin{bmatrix} \mathbf{A_1} & \mathbf{0} \\ \mathbf{0} & \mathbf{A_1 R} \end{bmatrix} \\
\mathbf{A_1} &= \mathbf{Q \Lambda Q}^{-1}, \\
\mathbf{R} &= \mathbf{Q \Phi Q}^{-1}, \\
\mathbf{\Phi} &= \text{diag}(e^{2j\pi\phi}, \ldots,  e^{2j\pi\phi})
\end{aligned}
where $\phi$ is a small, random phase term. This $\mathbf{A_2} = \mathbf{Q} (\mathbf{\Lambda} e^{2j\pi\phi}) \mathbf{Q}^{-1}$ is a small perturbation on $\mathbf{A_1}$.



In [ ]:
def fade_in_perturb_modes(rt60_ms: List, freq_hz: List, fs: float):
    """
    See fade in behaviour when we subtract modes with different decay rates
    and frequencies
    """
    ir_len_samps = ms_to_samps(ir_len_ms, fs)
    decay_slope = 1.0 / reverb_time_to_time_constant(rt60_ms, fs)
    time_vector = np.arange(0, float(ir_len_samps)/fs, 1.0/fs)
    assert len(freq_hz) == len(decay_slope)
    assert len(decay_slope) == 2, "This function only works for 2 slopes"

    omega = 2*np.pi*freq_hz
    output = np.sqrt(np.exp(2 * decay_slope[0] * time_vector) + np.exp(2 * decay_slope[1] * time_vector) - \
             2*np.exp(np.sum(decay_slope)*time_vector) * np.cos((omega[1] - omega[0])*time_vector))
    return time_vector, output


freq_hz = 1000
num_perturb = 5
perturb_hz = np.linspace(0, 10, num_perturb)
plt.figure()
for k in range(num_perturb):
    freq_hz_perturb = np.array([freq_hz, freq_hz + perturb_hz[k]])
    time_vector, output = fade_in_perturb_modes(rt60_ms, freq_hz_perturb, fs)
    plt.plot(time_vector, db(output), label=f'Perturb frequency: {perturb_hz[k]:.2f} Hz')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (dB)')
plt.title('EDC with varying modal frequency perturbation')
plt.ylim([-60, 0])
plt.legend()
plt.savefig(f'{fig_path}/fade_in_perturb_modes.png')

# slight perturbation in the two coupling matrices
A1 = get_random_unitary_matrix(num_delay_lines_per_group)
D, U = eig(A1)
eigenval_ang = np.linspace(0, 1, num_perturb)
plt.figure()

for k in range(num_perturb):
    eigenvals = np.ones(num_delay_lines_per_group) * np.exp(1j * 2 * np.pi * eigenval_ang[k])
    R = U @ np.diag(eigenvals) @ np.linalg.inv(U)
    A2 = A1 @ R 
    assert np.allclose(np.conj(A2.T) @ A2, np.eye(num_delay_lines_per_group))
    A_block = np.zeros((num_del_lines, num_del_lines), dtype=np.complex64)
    A_block[:num_delay_lines_per_group, :num_delay_lines_per_group] = A1
    A_block[num_delay_lines_per_group:, num_delay_lines_per_group:] = A2
            
    time_vector, output = simple_gfdn_impulse_response(taus, A_block, b, c, rt60_ms_exp, ir_len_ms, fs)
    envelope = calculate_energy_envelope(output, fs, smooth_time_ms=50)
    plt.plot(time_vector, db(envelope), label=f'Perturb frequency={eigenval_ang[k]:.2f} Hz')
    
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (dB)')
plt.title('RIR with asymmetric block diagonal feedback matrix')
plt.legend()
plt.savefig(f'{fig_path}/fade_in_perturb_feedback_matrix.png')

We also observe fade-in behaviour when the feedback matrix has inter-group coupling. In the first few echoes, when the effect of the feedback matrix has not kicked in, we essentially subtract the output of the two groups, which leads to fade-in behaviour. After some time, when the echoes traverse through the feedback matrix, the resulting RIR has a more complex EDC with two-stage decay. Let's see how the EDC changes as we vary the coupling from minimum to maximum

Is this also observed in real rooms? i.e, when there is no LOS between the source and the receiver, initially there is fade-in, but with time the EDC exhibits multi-stage decay (or does the slope of the room where the receiver is placed dominate the EDC?)

In [ ]:
def get_coupled_feedback_matrix(M: List[NDArray], coupling_angle:float) -> NDArray:
    """Get scalar coupled feedback matrix for 2 groups in GFDN"""
    assert M[0].shape == M[1].shape
    assert [is_unitary(torch.from_numpy(M[i]))[0] for i in range(len(M))]
    assert  0 <= coupling_angle <= np.pi/4

    N = M[0].shape[0]
    M_block = np.zeros((2*N, 2*N), dtype=M[1].dtype)
    # block diagonals
    M_block[:N, :N] = np.cos(coupling_angle) * (M[0] @ M[0])
    M_block[N:, N:] = np.cos(coupling_angle) * (M[1] @ M[1])
    # off-diagonals
    M_block[:N, N:] = -np.sin(coupling_angle) * (M[0] @ M[1])
    M_block[N:, :N] = np.sin(coupling_angle) * (M[1] @ M[0])
    assert np.allclose(M_block.T @ M_block, np.eye(2*N))
    return M_block

# try with some - inter-group coupling
M0 = A1 #get_random_unitary_matrix(num_delay_lines_per_group)
M1 = A2 #get_random_unitary_matrix(num_delay_lines_per_group)

num_angles = 10
coupling_angle = np.linspace(0, 1, num_angles) * np.pi/4

plt.figure()
for i in range(num_angles):
    M_block = get_coupled_feedback_matrix([M0, M1], coupling_angle[i])
    time_vector, output = simple_gfdn_impulse_response(taus, M_block, b, c, rt60_ms_exp, ir_len_ms, fs)
    envelope = calculate_energy_envelope(output, fs, smooth_time_ms=50)
    plt.plot(time_vector, db(envelope), label=f'phi={coupling_angle[i]/(0.25*np.pi):.2f}')


plt.xlabel('Time (s)')
plt.ylabel('Amplitude (dB)')
plt.legend()
plt.title('RIR with different amounts of coupling')
plt.savefig(f'{fig_path}/fade_in_vary_coupling.png')
